# Scientific American

Scrapes all issues and ebooks from Scientific American.

**Scraper needs an update that will enable it to notice a duplicated issue or ebook while it is scraping, and it will quit scraping any further pages.**

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import configparser
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import re
import pandas as pd

configs = configparser.ConfigParser()
configs.read('../config.ini')

['../config.ini']

## Define scraping functions:

In [98]:
def initialize(download_dir):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': download_dir,
             'plugins.always_open_pdf_externally': True}
    options.add_experimental_option('prefs', prefs)
    return webdriver.Chrome(options=options, executable_path='../chromedriver')

def login(username, password):
    signin_button_present = EC.presence_of_element_located((By.ID, 'signin-click'))
    wait.until(signin_button_present)
    signin_button = browser.find_element(By.ID, 'signin-click')
    signin_button.click()
    email_input = browser.find_element(By.ID, 'emailAddress1')
    password_input = browser.find_element(By.ID, 'password1')
    browser.execute_script('arguments[0].value = "' + username, email_input) 
    browser.execute_script('arguments[0].value = "' + password + '";', password_input)
    login_button = browser.find_element_by_class_name('signin-flyout__form__submit')
    time.sleep(2)
    login_button.click()
    signed_in = EC.invisibility_of_element_located((By.CLASS_NAME, 'signin-flyout__form__loading'))
    wait.until(signed_in)
    
def scrape_issue_links():
    view_details_links =  [x.get_attribute('href') for x in browser.find_elements_by_class_name('store-listing__cta')]
    for btn in view_details_links:
        browser.get(btn)
        link = browser.find_element_by_class_name('download-issue-btn-counter').get_attribute('href')
        # If link is already in previous data, quit
        # Add code here
        meta = re.search('Volume (\d+), Issue (\d+s?)', browser.find_element_by_class_name('product-detail__meta').text)
        volume = meta[1]
        issue = meta[2]
        date = browser.find_element_by_class_name('product-detail__title').text
        category = re.search('(CURRENT ISSUE)?(.+)', browser.find_element_by_class_name('product-detail__category').text)[2]
        data['category'].append(category)
        data['date'].append(date)
        data['volume'].append(volume)
        data['issue'].append(issue)
        data['link'].append(link)
        # Maybe return a boolean (all successful, false - had duplicates, then quit)
    
def scrape_ebook_links():
    view_details_links =  [x.get_attribute('href') for x in browser.find_elements_by_class_name('store-listing__cta')]
    for idx, btn in enumerate(view_details_links):
        browser.get(btn)
        title = browser.find_element_by_class_name('product-detail__title').text 
        data['title'].append(title)
        try:
            epub_link = browser.find_element_by_xpath('//*[contains(text(), "Download Epub/Other")]').get_attribute('href')
            mobi_link = browser.find_element_by_xpath('//*[contains(text(), "Download Mobi/Kindle")]').get_attribute('href')
            summary = browser.find_elements_by_class_name('product-detail__meta')[1].find_element_by_tag_name('p').text        
            date = browser.find_elements_by_class_name('product-detail__meta')[1].text.split('On Sale Date: ')[1]
            data['date'].append(date)
            data['epub_link'].append(epub_link) 
            data['mobi_link'].append(mobi_link)
            data['pdf_link'].append(None)
            data['summary'].append(summary)
        except NoSuchElementException:
            print('NoSuchElementException...')
            pdf_link = browser.find_element_by_xpath('//*[contains(text(), "Download")]').get_attribute('href')
            summary = browser.find_element_by_class_name('product-detail__body').text
            data['epub_link'].append(None) 
            data['mobi_link'].append(None)
            data['pdf_link'].append(pdf_link)
            data['summary'].append(summary)
            data['date'].append(None)

## Log in, navigating to issue archive

Log into account.

In [177]:
# Initialize browser
download_path = '/Volumes/ARCHIVES/GitHub/scientific-american/issues'
browser = initialize(download_path)
wait = WebDriverWait(browser, 15)

# Navigate to page with search results for issues
current_url = 'https://www.scientificamerican.com/store/archive/?page=1'
browser.get(current_url)

# Log into my account 
login(configs['SCIAM']['EMAIL'] + '";', configs['SCIAM']['PASSWORD'])

TimeoutException: Message: 


## Collect download links for magazines 

**Add code here that reads in issues.csv. Update the scraper function above to check this data for duplicates. Then update the code below to concatenate the two data frames, the new one, and the old one, after scraping, and update issues.csv**

Scrape.

In [100]:
data = {'category': [], 
        'volume': [],
        'issue': [],
        'date': [],
        'link': []}

# Collect all download links on the first page
total_pages = int(browser.find_elements_by_class_name('pagination__nums__item')[-1].find_element_by_tag_name('a').text)
pages_left = total_pages - 1
current_page = 2
print('Scraping page 1/' + str(total_pages) + '...')
scrape_issue_links()

# Paginate until the end, scraping all download links
while pages_left > 0: 
    try: 
        browser.get('https://www.scientificamerican.com/store/archive/?page=' + str(current_page))
        current_total_pages = int(browser.find_elements_by_class_name('pagination__nums__item')[-1].find_element_by_tag_name('a').text)
        print('Scraping page ' + str(current_page) + '/' + str(total_pages) + '...')
        scrape_issue_links()
        if total_pages != current_total_pages:
            print('Detected change in total pages.')
            pages_left = current_total_pages - (total_pages - pages_left)
            total_pages = current_total_pages
        current_page += 1
        pages_left -= 1
        time.sleep(5) # according to robots.txt
    except IndexError: # Your connection was interrupted. 
        print('Connection interrupted, reconnecting...')
        continue

Scraping page 1/441...
Scraping page 2/441...
Scraping page 3/441...
Scraping page 4/441...
Scraping page 5/441...
Scraping page 6/441...
Scraping page 7/441...
Scraping page 8/441...
Scraping page 9/441...
Scraping page 10/441...
Scraping page 11/441...
Scraping page 12/441...
Scraping page 13/441...
Scraping page 14/441...
Scraping page 15/441...
Scraping page 16/441...
Scraping page 17/441...
Scraping page 18/441...
Scraping page 19/441...
Scraping page 20/441...
Scraping page 21/441...
Scraping page 22/441...
Scraping page 23/441...
Scraping page 24/441...
Scraping page 25/441...
Scraping page 26/441...
Scraping page 27/441...
Scraping page 28/441...
Scraping page 29/441...
Scraping page 30/441...
Scraping page 31/441...
Scraping page 32/441...
Scraping page 33/441...
Scraping page 34/441...
Scraping page 35/441...
Scraping page 36/441...
Scraping page 37/441...
Scraping page 38/441...
Scraping page 39/441...
Scraping page 40/441...
Scraping page 41/441...
Scraping page 42/441...
S

Scraping page 334/441...
Scraping page 335/441...
Scraping page 336/441...
Scraping page 337/441...
Scraping page 338/441...
Scraping page 339/441...
Scraping page 340/441...
Scraping page 341/441...
Scraping page 342/441...
Scraping page 343/441...
Scraping page 344/441...
Scraping page 345/441...
Scraping page 346/441...
Scraping page 347/441...
Scraping page 348/441...
Scraping page 349/441...
Scraping page 350/441...
Scraping page 351/441...
Scraping page 352/441...
Scraping page 353/441...
Scraping page 354/441...
Scraping page 355/441...
Scraping page 356/441...
Scraping page 357/441...
Scraping page 358/441...
Scraping page 359/441...
Scraping page 360/441...
Scraping page 361/441...
Scraping page 362/441...
Scraping page 363/441...
Scraping page 364/441...
Scraping page 365/441...
Scraping page 366/441...
Scraping page 367/441...
Scraping page 368/441...
Scraping page 369/441...
Scraping page 370/441...
Scraping page 371/441...
Scraping page 372/441...
Scraping page 373/441...


### Clean magazine data.

In [136]:
issues_df = pd.DataFrame(data)
issues_df

,category,volume,issue,date,link
0,SCIENTIFIC AMERICAN SPACE & PHYSICS,3,6,December 2020,https://www.scientificamerican.com/index.cfm/_...
1,SCIENTIFIC AMERICAN HEALTH & MEDICINE,2,6,December 2020,https://www.scientificamerican.com/index.cfm/_...
2,SCIENTIFIC AMERICAN,323,6,December 2020,https://www.scientificamerican.com/index.cfm/_...
3,SCIENTIFIC AMERICAN,323,5,November 2020,https://www.scientificamerican.com/index.cfm/_...
4,SCIENTIFIC AMERICAN MIND,31,6,November 2020,https://www.scientificamerican.com/index.cfm/_...
...,...,...,...,...,...
5287,SCIENTIFIC AMERICAN,1,5,"September 25, 1845",https://www.scientificamerican.com/index.cfm/_...
5288,SCIENTIFIC AMERICAN,1,4,"September 18, 1845",https://www.scientificamerican.com/index.cfm/_...
5289,SCIENTIFIC AMERICAN,1,3,"September 11, 1845",https://www.scientificamerican.com/index.cfm/_...
5290,SCIENTIFIC AMERICAN,1,2,"September 04, 1845",https://www.scientificamerican.com/index.cfm/_...


Check duplicates.

In [137]:
dupes = issues_df[issues_df['link'].duplicated(keep=False)]
dupes

,category,volume,issue,date,link
922,SCIENTIFIC AMERICAN,200,1,January 1959,https://www.scientificamerican.com/index.cfm/_...
923,SCIENTIFIC AMERICAN,200,1,January 1959,https://www.scientificamerican.com/index.cfm/_...
1607,SCIENTIFIC AMERICAN,116,13,"March 31, 1917",https://www.scientificamerican.com/index.cfm/_...
1608,SCIENTIFIC AMERICAN,116,13,"March 31, 1917",https://www.scientificamerican.com/index.cfm/_...
4055,SCIENTIFIC AMERICAN,21,19,"November 06, 1869",https://www.scientificamerican.com/index.cfm/_...
4056,SCIENTIFIC AMERICAN,21,19,"November 06, 1869",https://www.scientificamerican.com/index.cfm/_...


In [138]:
issues_df.drop_duplicates(inplace=True)
print('Dropped', int(len(dupes)/2), 'duplicates')

Dropped 3 duplicates


Check categories.

In [139]:
issues_df['category'].value_counts()

SCIENTIFIC AMERICAN                       5153
SCIENTIFIC AMERICAN MIND                   106
SCIENTIFIC AMERICAN SPACE & PHYSICS         16
SCIENTIFIC AMERICAN HEALTH & MEDICINE       11
 SCIENTIFIC AMERICAN                         1
 SCIENTIFIC AMERICAN SPACE & PHYSICS         1
 SCIENTIFIC AMERICAN HEALTH & MEDICINE       1
Name: category, dtype: int64

In [141]:
issues_df['category'] = issues_df['category'].apply(lambda x: x.strip())
issues_df['category'].value_counts()

SCIENTIFIC AMERICAN                      5154
SCIENTIFIC AMERICAN MIND                  106
SCIENTIFIC AMERICAN SPACE & PHYSICS        17
SCIENTIFIC AMERICAN HEALTH & MEDICINE      12
Name: category, dtype: int64

In [308]:
issues_df.to_csv('issues.csv', index=False)

## Collect download links for ebooks.

In [132]:
# Set up data
data = {'title': [], 
        'date': [],
        'summary': [],
        'pdf_link': [],
        'epub_link': [],
        'mobi_link': []}

# Navigate to URL for ebooks
current_url = 'https://www.scientificamerican.com/store/ebooks/'
browser.get(current_url)

# Collect all download links on the first page
total_pages = int(browser.find_elements_by_class_name('pagination__nums__item')[-1].find_element_by_tag_name('a').text)
pages_left = total_pages - 1
current_page = 2
print('Scraping page 1/' + str(total_pages) + '...')
scrape_ebook_links()

# Paginate until the end, scraping all download links
while pages_left > 0: 
    try: 
        browser.get('https://www.scientificamerican.com/store/ebooks/?page=' + str(current_page))
        current_total_pages = int(browser.find_elements_by_class_name('pagination__nums__item')[-1].find_element_by_tag_name('a').text)
        print('Scraping page ' + str(current_page) + '/' + str(total_pages) + '...')
        scrape_ebook_links()      
        if total_pages != current_total_pages:
            print('Detected change in total pages.')
            pages_left = current_total_pages - (total_pages - pages_left)
            total_pages = current_total_pages
        current_page += 1
        pages_left -= 1
        time.sleep(5) # according to robots.txt
    except IndexError: # Your connection was interrupted. 
        print('Connection interrupted, reconnecting...')
        continue

Scraping page 1/9...
Scraping page 2/9...
Scraping page 3/9...
Scraping page 4/9...
NoSuchElementException...
Scraping page 5/9...
Scraping page 6/9...
Scraping page 7/9...
Scraping page 8/9...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
Scraping page 9/9...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...
NoSuchElementException...


Save ebooks data.

In [133]:
ebooks_df = pd.DataFrame(data)
ebooks_df

,title,date,summary,pdf_link,epub_link,mobi_link
0,Hacking the Immune System,11/2/20,"The immune system is a marvel, but sometimes t...",None,https://www.scientificamerican.com/index.cfm/_...,https://www.scientificamerican.com/index.cfm/_...
1,Technology vs. Truth: Deception in the Digital...,10/5/20,"In the digital age, information, both true and...",None,https://www.scientificamerican.com/index.cfm/_...,https://www.scientificamerican.com/index.cfm/_...
2,"Stressed Out: Causes, Effects and Keeping Calm",8/31/20,Chronic stress makes people sick. The fight-or...,None,https://www.scientificamerican.com/index.cfm/_...,https://www.scientificamerican.com/index.cfm/_...
3,Black Holes: Going to Extremes,8/3/20,"Once dismissed as a mathematical curiosity, bl...",None,https://www.scientificamerican.com/index.cfm/_...,https://www.scientificamerican.com/index.cfm/_...
4,Quantum Universe,7/6/20,"Strange and probabilistic, physics at the smal...",None,https://www.scientificamerican.com/index.cfm/_...,https://www.scientificamerican.com/index.cfm/_...
...,...,...,...,...,...,...
103,Nature Books and Arts Special: 2014. No 1,None,A collection of book reviews for the scientifi...,https://www.scientificamerican.com/index.cfm/_...,None,None
104,Books & Arts Special 2015 No.1,None,In Nature’s first book reviews supplement for ...,https://www.scientificamerican.com/index.cfm/_...,None,None
105,Nature Collections: Stem Cells – Breaking Barr...,None,The most versatile of stem cells can be made f...,https://www.scientificamerican.com/index.cfm/_...,None,None
106,Nature Collections: Homo Floresiensis – the ‘h...,None,The remains of the tiny hominin Homo floresien...,https://www.scientificamerican.com/index.cfm/_...,None,None


In [134]:
ebooks_df.to_csv('ebooks.csv', index=False)

## Download all issues and ebooks.

### Download issues. 

In [310]:
# Initialize browser
download_path = '/Volumes/ARCHIVES/GitHub/scientific-american/issues'
browser = initialize(download_path)
wait = WebDriverWait(browser, 15)

# Navigate to page with search results for issues
current_url = 'https://www.scientificamerican.com/store/archive/?page=1'
browser.get(current_url)

# Log into my account 
login(configs['SCIAM']['EMAIL'] + '";', configs['SCIAM']['PASSWORD'])

# Get downloads page
browser.get("about:downloads")

In [311]:
download_links = issues_df['link']
print(len(download_links), 'issues to download...')
downloaded_filenames = []

def download(issue):
    if issue['downloaded_filename']:
        print('Already downloaded', issue['date'], ', Volume', issue['volume'], 'Issue', issue['issue'], 'to', issue['downloaded_filename'])
        return
    browser.get(issue['link']) # download issue
    time.sleep(10) # pause so that we don't download too many at once
    filename = browser.execute_script("return document.querySelector('downloads-manager').shadowRoot.querySelector('#downloadsList downloads-item').shadowRoot.querySelector('#title-area')")  
    downloaded_filenames.append(filename.text)
    print('Downloaded', issue['date'], ', Volume', issue['volume'], 'Issue', issue['issue'], 'to', filename.text)
    
issues_df.apply(download, axis=1)

5289 issues to download...
Already downloaded December 2020 , Volume 3 Issue 6 to SP&P_DecJan.pdf
Already downloaded December 2020 , Volume 2 Issue 6 to SA_H&M_DEC2020.pdf
Already downloaded December 2020 , Volume 323 Issue 6 to 2020_12.pdf
Already downloaded November 2020 , Volume 323 Issue 5 to 2020_11.pdf
Already downloaded November 2020 , Volume 31 Issue 6 to sa_MIND_NovDec.pdf
Already downloaded October 2020 , Volume 29 Issue 4s to OSQ420_Truth.pdf
Already downloaded October 2020 , Volume 3 Issue 5 to SP&POCTNOV2020.pdf
Already downloaded October 2020 , Volume 323 Issue 4 to 2020_10.pdf
Already downloaded October 2020 , Volume 2 Issue 5 to H&M_OCTNOV.pdf
Already downloaded September 2020 , Volume 31 Issue 5 to sa_MIND_septoct.pdf
Already downloaded September 2020 , Volume 323 Issue 3 to 2020_09.pdf
Already downloaded August 2020 , Volume 29 Issue 3s to OSQ320_ClimateChange.pdf
Already downloaded August 2020 , Volume 3 Issue 4 to SA-S&P-AUG-SEPT.pdf
Already downloaded August 2020 ,

Already downloaded November 2011 , Volume 22 Issue 5 to SAMind_201111.pdf
Already downloaded November 2011 , Volume 305 Issue 5 to 2011-11-01.pdf
Already downloaded October 2011 , Volume 305 Issue 4 to 2011-10-01.pdf
Already downloaded September 2011 , Volume 22 Issue 4 to SAMind_201109.pdf
Already downloaded September 2011 , Volume 305 Issue 3 to 2011-09-01.pdf
Already downloaded August 2011 , Volume 305 Issue 2 to 2011-08-01.pdf
Already downloaded July 2011 , Volume 22 Issue 3 to SAMind_201107.pdf
Already downloaded July 2011 , Volume 305 Issue 1 to 2011-07-01.pdf
Already downloaded June 2011 , Volume 304 Issue 6 to 2011-06-01.pdf
Already downloaded May 2011 , Volume 304 Issue 5 to 2011-05-01.pdf
Already downloaded May 2011 , Volume 22 Issue 2 to SAMind_201105.pdf
Already downloaded April 2011 , Volume 304 Issue 4 to 2011-04-01.pdf
Already downloaded March 2011 , Volume 22 Issue 1 to SAMind_201103.pdf
Already downloaded March 2011 , Volume 304 Issue 3 to 2011-03-01.pdf
Already downlo

Already downloaded June 2005 , Volume 16 Issue 2 to SAMind_200506.pdf
Already downloaded May 2005 , Volume 292 Issue 5 to 2005-05-01.pdf
Already downloaded April 2005 , Volume 292 Issue 4 to 2005-04-01.pdf
Already downloaded April 2005 , Volume 16 Issue 1 to SAMind_200504.pdf
Already downloaded March 2005 , Volume 292 Issue 3 to 2005-03-01.pdf
Already downloaded February 2005 , Volume 292 Issue 2 to 2005-02-01.pdf
Already downloaded January 2005 , Volume 292 Issue 1 to 2005-01-01.pdf
Already downloaded January 2005 , Volume 15 Issue 1s to 200501.pdf
Already downloaded December 2004 , Volume 291 Issue 6 to 2004-12-01.pdf
Already downloaded December 2004 , Volume 14 Issue 5 to SAMind_200412.pdf
Already downloaded November 2004 , Volume 291 Issue 5 to 2004-11-01.pdf
Already downloaded October 2004 , Volume 291 Issue 4 to 2004-10-01.pdf
Already downloaded September 2004 , Volume 14 Issue 4s to 200409.pdf
Already downloaded September 2004 , Volume 291 Issue 3 to 2004-09-01.pdf
Already downl

Already downloaded December 1995 , Volume 273 Issue 6 to 1995-12-01.pdf
Already downloaded November 1995 , Volume 273 Issue 5 to 1995-11-01.pdf
Already downloaded October 1995 , Volume 273 Issue 4 to 1995-10-01.pdf
Already downloaded September 1995 , Volume 273 Issue 3 to 1995-09-01.pdf
Already downloaded August 1995 , Volume 273 Issue 2 to 1995-08-01.pdf
Already downloaded July 1995 , Volume 273 Issue 1 to 1995-07-01.pdf
Already downloaded June 1995 , Volume 272 Issue 6 to 1995-06-01.pdf
Already downloaded May 1995 , Volume 272 Issue 5 to 1995-05-01.pdf
Already downloaded April 1995 , Volume 272 Issue 4 to 1995-04-01.pdf
Already downloaded March 1995 , Volume 272 Issue 3 to 1995-03-01.pdf
Already downloaded February 1995 , Volume 272 Issue 2 to 1995-02-01.pdf
Already downloaded January 1995 , Volume 272 Issue 1 to 1995-01-01.pdf
Already downloaded December 1994 , Volume 271 Issue 6 to 1994-12-01.pdf
Already downloaded November 1994 , Volume 271 Issue 5 to 1994-11-01.pdf
Already downlo

Already downloaded July 1985 , Volume 253 Issue 1 to 1985-07-01.pdf
Already downloaded June 1985 , Volume 252 Issue 6 to 1985-06-01.pdf
Already downloaded May 1985 , Volume 252 Issue 5 to 1985-05-01.pdf
Already downloaded April 1985 , Volume 252 Issue 4 to 1985-04-01.pdf
Already downloaded March 1985 , Volume 252 Issue 3 to 1985-03-01.pdf
Already downloaded February 1985 , Volume 252 Issue 2 to 1985-02-01.pdf
Already downloaded January 1985 , Volume 252 Issue 1 to 1985-01-01.pdf
Already downloaded December 1984 , Volume 251 Issue 6 to 1984-12-01.pdf
Already downloaded November 1984 , Volume 251 Issue 5 to 1984-11-01.pdf
Already downloaded October 1984 , Volume 251 Issue 4 to 1984-10-01.pdf
Already downloaded September 1984 , Volume 251 Issue 3 to 1984-09-01.pdf
Already downloaded August 1984 , Volume 251 Issue 2 to 1984-08-01.pdf
Already downloaded July 1984 , Volume 251 Issue 1 to 1984-07-01.pdf
Already downloaded June 1984 , Volume 250 Issue 6 to 1984-06-01.pdf
Already downloaded May

...

Downloaded July 29, 1854 , Volume 9 Issue 46 to 1854-07-29.pdf
Downloaded July 22, 1854 , Volume 9 Issue 45 to 1854-07-22.pdf
Downloaded July 15, 1854 , Volume 9 Issue 44 to 1854-07-15.pdf
Downloaded July 08, 1854 , Volume 9 Issue 43 to 1854-07-08.pdf
Downloaded July 01, 1854 , Volume 9 Issue 42 to 1854-07-01.pdf
Downloaded June 24, 1854 , Volume 9 Issue 41 to 1854-06-24.pdf
Downloaded June 17, 1854 , Volume 9 Issue 40 to 1854-06-17.pdf
Downloaded June 10, 1854 , Volume 9 Issue 39 to 1854-06-10.pdf
Downloaded June 03, 1854 , Volume 9 Issue 38 to 1854-06-03.pdf
Downloaded May 27, 1854 , Volume 9 Issue 37 to 1854-05-27.pdf
Downloaded May 20, 1854 , Volume 9 Issue 36 to 1854-05-20.pdf
Downloaded May 13, 1854 , Volume 9 Issue 35 to 1854-05-13.pdf
Downloaded May 06, 1854 , Volume 9 Issue 34 to 1854-05-06.pdf
Downloaded April 29, 1854 , Volume 9 Issue 33 to 1854-04-29.pdf
Downloaded April 22, 1854 , Volume 9 Issue 32 to 1854-04-22.pdf
Downloaded April 15, 1854 , Volume 9 Issue 31 to 1854-04-

Downloaded January 10, 1852 , Volume 7 Issue 17 to 1852-01-10.pdf
Downloaded January 03, 1852 , Volume 7 Issue 16 to 1852-01-03.pdf
Downloaded December 27, 1851 , Volume 7 Issue 15 to 1851-12-27.pdf
Downloaded December 20, 1851 , Volume 7 Issue 14 to 1851-12-20.pdf
Downloaded December 13, 1851 , Volume 7 Issue 13 to 1851-12-13.pdf
Downloaded December 06, 1851 , Volume 7 Issue 12 to 1851-12-06.pdf
Downloaded November 29, 1851 , Volume 7 Issue 11 to 1851-11-29.pdf
Downloaded November 22, 1851 , Volume 7 Issue 10 to 1851-11-22.pdf
Downloaded November 15, 1851 , Volume 7 Issue 9 to 1851-11-15.pdf
Downloaded November 08, 1851 , Volume 7 Issue 8 to 1851-11-08.pdf
Downloaded November 01, 1851 , Volume 7 Issue 7 to 1851-11-01.pdf
Downloaded October 25, 1851 , Volume 7 Issue 6 to 1851-10-25.pdf
Downloaded October 18, 1851 , Volume 7 Issue 5 to 1851-10-18.pdf
Downloaded October 11, 1851 , Volume 7 Issue 4 to 1851-10-11.pdf
Downloaded October 04, 1851 , Volume 7 Issue 3 to 1851-10-04.pdf
Download

Downloaded August 11, 1849 , Volume 4 Issue 47 to 1849-08-11.pdf
Downloaded August 04, 1849 , Volume 4 Issue 46 to 1849-08-04.pdf
Downloaded July 28, 1849 , Volume 4 Issue 45 to 1849-07-28.pdf
Downloaded July 21, 1849 , Volume 4 Issue 44 to 1849-07-21.pdf
Downloaded July 14, 1849 , Volume 4 Issue 43 to 1849-07-14.pdf
Downloaded July 07, 1849 , Volume 4 Issue 42 to 1849-07-07.pdf
Downloaded June 30, 1849 , Volume 4 Issue 41 to 1849-06-30.pdf
Downloaded June 23, 1849 , Volume 4 Issue 40 to 1849-06-23.pdf
Downloaded June 16, 1849 , Volume 4 Issue 39 to 1849-06-16.pdf
Downloaded June 09, 1849 , Volume 4 Issue 38 to 1849-06-09.pdf
Downloaded June 02, 1849 , Volume 4 Issue 37 to 1849-06-02.pdf
Downloaded May 26, 1849 , Volume 4 Issue 36 to 1849-05-26.pdf
Downloaded May 19, 1849 , Volume 4 Issue 35 to 1849-05-19.pdf
Downloaded May 12, 1849 , Volume 4 Issue 34 to 1849-05-12.pdf
Downloaded May 05, 1849 , Volume 4 Issue 33 to 1849-05-05.pdf
Downloaded April 28, 1849 , Volume 4 Issue 32 to 1849-0

Downloaded March 06, 1847 , Volume 2 Issue 24 to 1847-03-06.pdf
Downloaded February 27, 1847 , Volume 2 Issue 23 to 1847-02-27.pdf
Downloaded February 20, 1847 , Volume 2 Issue 22 to 1847-02-20.pdf
Downloaded February 13, 1847 , Volume 2 Issue 21 to 1847-02-13.pdf
Downloaded February 06, 1847 , Volume 2 Issue 20 to 1847-02-06.pdf
Downloaded January 30, 1847 , Volume 2 Issue 19 to 1847-01-30.pdf
Downloaded January 23, 1847 , Volume 2 Issue 18 to 1847-01-23.pdf
Downloaded January 16, 1847 , Volume 2 Issue 17 to 1847-01-16.pdf
Downloaded January 09, 1847 , Volume 2 Issue 16 to 1847-01-09.pdf
Downloaded January 02, 1847 , Volume 2 Issue 15 to 1847-01-02.pdf
Downloaded December 26, 1846 , Volume 2 Issue 14 to 1846-12-26.pdf
Downloaded December 19, 1846 , Volume 2 Issue 13 to 1846-12-19.pdf
Downloaded December 12, 1846 , Volume 2 Issue 12 to 1846-12-12.pdf
Downloaded December 05, 1846 , Volume 2 Issue 11 to 1846-12-05.pdf
Downloaded November 28, 1846 , Volume 2 Issue 10 to 1846-11-28.pdf
Dow

0       None
1       None
2       None
3       None
4       None
        ... 
5287    None
5288    None
5289    None
5290    None
5291    None
Length: 5289, dtype: object

Will need to add a way to insert these newly downloaded_filenames after the ones that are already in the issues_df.

Insert those newly downloaded filenames between...

In [ ]:
lo = len(issues_df['downloaded_filename'].dropna())
hi = lo + len(downloaded_filenames)
nones = []
if hi < len(issues_df): # if we don't have all filenames yet we need to generate Nones
    nones = [None for i in range(0, len(issues_df) - hi)]
issues_df['downloaded_filename'] = list(issues_df.iloc[:lo]['downloaded_filename']) + downloaded_filenames + nones

In [339]:
issues_df.to_csv('issues.csv', index=None)
issues_df

,category,volume,issue,date,link,downloaded_filename
0,SCIENTIFIC AMERICAN SPACE & PHYSICS,3,6,December 2020,https://www.scientificamerican.com/index.cfm/_...,SP&P_DecJan.pdf
1,SCIENTIFIC AMERICAN HEALTH & MEDICINE,2,6,December 2020,https://www.scientificamerican.com/index.cfm/_...,SA_H&M_DEC2020.pdf
2,SCIENTIFIC AMERICAN,323,6,December 2020,https://www.scientificamerican.com/index.cfm/_...,2020_12.pdf
3,SCIENTIFIC AMERICAN,323,5,November 2020,https://www.scientificamerican.com/index.cfm/_...,2020_11.pdf
4,SCIENTIFIC AMERICAN MIND,31,6,November 2020,https://www.scientificamerican.com/index.cfm/_...,sa_MIND_NovDec.pdf
...,...,...,...,...,...,...
5287,SCIENTIFIC AMERICAN,1,5,"September 25, 1845",https://www.scientificamerican.com/index.cfm/_...,1845-09-25.pdf
5288,SCIENTIFIC AMERICAN,1,4,"September 18, 1845",https://www.scientificamerican.com/index.cfm/_...,1845-09-18.pdf
5289,SCIENTIFIC AMERICAN,1,3,"September 11, 1845",https://www.scientificamerican.com/index.cfm/_...,1845-09-11.pdf
5290,SCIENTIFIC AMERICAN,1,2,"September 04, 1845",https://www.scientificamerican.com/index.cfm/_...,1845-09-04.pdf


### Download ebooks.